In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [2]:
# 데이터 로드

train = pd.read_parquet('./train_pwm.parquet')
train.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,weekend,month
0,TRAIN_0000000,20220623,1,17,1,106,지방도1112호선,0,0,60.0,...,33.427747,126.662612,0,제3교래교,33.427749,126.662335,0,52.0,0,6
1,TRAIN_0000001,20220728,1,21,2,103,일반국도11호선,0,0,60.0,...,33.500730,126.529107,1,KAL사거리,33.504811,126.526240,0,30.0,0,7
2,TRAIN_0000002,20211010,4,7,2,103,일반국도16호선,0,0,80.0,...,33.279145,126.368598,0,상창육교,33.280072,126.362147,0,61.0,1,10
3,TRAIN_0000003,20220311,0,13,2,107,태평로,0,0,50.0,...,33.246081,126.567204,0,서현주택,33.245565,126.566228,0,20.0,0,3
4,TRAIN_0000004,20211005,6,8,2,103,일반국도12호선,0,0,80.0,...,33.462214,126.326551,0,애월입구,33.462677,126.330152,0,38.0,0,10


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 25 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   base_date              int64  
 2   day_of_week            int32  
 3   base_hour              int64  
 4   lane_count             int64  
 5   road_rating            int64  
 6   road_name              object 
 7   multi_linked           int64  
 8   connect_code           int64  
 9   maximum_speed_limit    float64
 10  vehicle_restricted     float64
 11  weight_restricted      float64
 12  height_restricted      float64
 13  road_type              int64  
 14  start_node_name        object 
 15  start_latitude         float64
 16  start_longitude        float64
 17  start_turn_restricted  int32  
 18  end_node_name          object 
 19  end_latitude           float64
 20  end_longitude          float64
 21  end_turn_restricted    int32  
 22  target            

In [3]:
from sklearn.utils import shuffle

tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

C:\Users\soy66\AppData\Local\Temp\ipykernel_3340\489013793.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
C:\Users\soy66\AppData\Local\Temp\ipykernel_3340\489013793.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


(4426522, 14)
(4426522,)
(274695, 14)


In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV



LR = lgb.LGBMRegressor()


parameters = {'max_depth':[10,15,20],
              'num_leaves':[150, 250,300],
              'learning_rate' : [0.3,0.5, 0.6]}


grid_lr = GridSearchCV(LR, param_grid=parameters, cv=5)


grid_lr.fit(X_train, y_train)


scores_df = pd.DataFrame(grid_lr.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'learning_rate': 0.3, 'max_depth': 10, 'num_l...",0.909277,27,0.909205,0.908574,0.910493
1,"{'learning_rate': 0.3, 'max_depth': 10, 'num_l...",0.912347,22,0.912223,0.911892,0.913220
2,"{'learning_rate': 0.3, 'max_depth': 10, 'num_l...",0.913143,17,0.912890,0.912803,0.913885
3,"{'learning_rate': 0.3, 'max_depth': 15, 'num_l...",0.910537,26,0.909965,0.910465,0.911446
4,"{'learning_rate': 0.3, 'max_depth': 15, 'num_l...",0.913632,16,0.913176,0.913229,0.914577
5,"{'learning_rate': 0.3, 'max_depth': 15, 'num_l...",0.914524,12,0.914377,0.914107,0.915322
6,"{'learning_rate': 0.3, 'max_depth': 20, 'num_l...",0.910662,25,0.910679,0.909987,0.911587
7,"{'learning_rate': 0.3, 'max_depth': 20, 'num_l...",0.913877,15,0.913697,0.913527,0.914831
8,"{'learning_rate': 0.3, 'max_depth': 20, 'num_l...",0.914642,11,0.914373,0.914219,0.915375
9,"{'learning_rate': 0.5, 'max_depth': 10, 'num_l...",0.911846,24,0.911738,0.911260,0.912567


In [6]:
grid_lr.cv_results_

{'mean_fit_time': array([25.1487555 , 30.0342999 , 33.01042433, 23.48860745, 27.98347883,
        32.05648661, 25.8241221 , 31.46123638, 33.95542464, 22.99105072,
        23.09700298, 25.87736421, 18.7711627 , 21.82639637, 23.85205674,
        18.76516976, 21.63629704, 22.91408086, 21.16408391, 28.38365192,
        26.83257618, 20.66978316, 21.25512023, 22.62366743, 18.41672473,
        20.94574513, 22.8052557 ]),
 'std_fit_time': array([2.62891805, 1.15504153, 2.54587672, 0.90293622, 1.77285882,
        0.52194867, 0.5695174 , 2.22128609, 1.52956506, 2.84193641,
        0.09081278, 0.69149028, 0.09079761, 0.50996337, 0.91256431,
        0.35696593, 0.31796294, 0.25002404, 2.20256779, 2.06583187,
        0.71635366, 0.58483092, 0.22390385, 0.21784582, 0.83962162,
        0.2186018 , 0.69750394]),
 'mean_score_time': array([2.74284039, 3.2780561 , 3.88279004, 2.64467158, 3.04642892,
        3.41631627, 2.85705328, 3.16183138, 3.37917986, 2.56284251,
        2.68381948, 2.87790041, 2.082

In [9]:
# grid_dtree.cv_results_
LR = lgb.LGBMRegressor(random_state = 50,
                           max_depth = 30,
                           num_leaves = 200,
                           learning_rate = 0.3,
                           n_estimators = 200,
                           
                          ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)
print(mae)
print(LR.feature_importances_)

3.6569229864691946
[11348  1405  1101  1186   407   231  5013  3790   193  3749  3072   151
  1709  6445]


In [11]:
importances = pd.Series(LR.feature_importances_, X_train.columns)
importances.sort_values()

end_turn_restricted        151
start_turn_restricted      193
road_type                  231
weight_restricted          407
road_rating               1101
maximum_speed_limit       1186
lane_count                1405
weekend                   1709
end_longitude             3072
end_latitude              3749
start_longitude           3790
start_latitude            5013
month                     6445
base_hour                11348
dtype: int32